# DIST-S1 Performance Assessment

Accuracy estimation for equal probability sampling, from Tyukavina et al. (2025) "Practical global sampling methods for estimating area and map accuracy of land cover and change" https://doi.org/10.1016/j.rse.2025.114714, Appendix A.1.1

Sample is a stratified sample of 10x10km blocks stratified by land cover change between 2023 and 2024. A ratio estimator is used. As pointed out in the Global Land Cover Map Validation Guidelines (https://doi.org/10.5067/doc/ceoswgcv/lpv/lc.001), the second stage variance typically contributes a negligible amount compared to the first stage variance and can be excluded.

#### Import sample and stratification information

In [1]:
# Import strata and sample unit information
import sys 
import math
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import statistics 
np.set_printoptions(precision=2, suppress=True, floatmode='fixed')

source = "/gpfs/glad3/HLSDIST/Validation/2024_10kmblock/analysis/"  # Replace with the desired path
os.chdir(source)

referenceSource = source+"tables/reference_data/referenceTimeSeriesInterpolated16_16_goodFirst.csv"
referenceSource = source+"tables/reference_data/referenceTimeSeriesInterpolated16_16_first.csv"
mapsourceHLS = "mapLabels2024"
mapsourceS1 = source+"generate_dist_s1_table/dist_s1_label_tables"
ANNname = "2024"
sampleDict = {}
sampleFull = {}
blockstrataDict = {}
substrataDict = {}

with open(source+"tables/reference_data/selectedpointsLL.csv","r") as sample:
  lines = sample.readlines()[1:]
  for l in lines:
    (ID,Block,subID,blockStratum,substratum,zone,x,y,centxUTM,centyUTM,Long,Lat,MGRS) = l.strip().split(",")
    sampleDict[ID] = ",".join([Long,Lat,zone,centxUTM,centyUTM])
    sampleFull[ID] = l.strip()
    blockstrataDict[Block] = blockStratum
    substrataDict[ID] = int(substratum)
allIDs = sampleDict.keys()

#Strata area
#scounts = allblocks['stratum'].value_counts()
scounts = pd.read_csv(source+"tables/reference_data/blockStrataCounts.csv").set_index('name')
scounts['area'] = scounts.multiply(100)
print(scounts)
allStrata = list(scounts.index)
totalBlockCount = scounts['blockCount'].sum()
print('totalBlockCount',totalBlockCount)
print(allStrata)

               blockCount       area
name                                
waternew             1272     127200
treelosswet          9696     969600
builtnewalert      120496   12049600
fire                 3188     318800
treelossTF          63161    6316100
cropnew             75912    7591200
wetshort             9477     947700
oldcrop_short       65470    6547000
gen                215057   21505700
other              385127   38512700
none              1085543  108554300
totalBlockCount 2034399
['waternew', 'treelosswet', 'builtnewalert', 'fire', 'treelossTF', 'cropnew', 'wetshort', 'oldcrop_short', 'gen', 'other', 'none']


In [2]:
selectedBlocks = pd.read_csv(source+"tables/reference_data/blockstrata_subareas.csv")
selectedBlocks = selectedBlocks.set_index('block')
print(selectedBlocks.head())

def getBlocksStratum(stratum):
 return list(selectedBlocks[selectedBlocks['stratum']==stratum].index)

def getBlockPixelCount(block):
 return selectedBlocks.loc[int(block)][['sub1','sub2','sub3','sub4']].sum()


        MGRS      stratum  sub1   sub2  sub3    sub4       left   top  \
block                                                                   
30961  33NUF   treelossTF  2263     67  2493  105873   13.80825  4.95   
34405  50NPL  treelosswet  1614   6024  2755  100467  118.01200  5.40   
35975  37NEG   treelossTF  1369    231  2864  106424   39.09600  5.67   
40284  47NRG  treelosswet   762   2115  7183  100894  101.90300  6.30   
41318  36NUN     waternew   873  10119  6819   92899   31.40350  6.48   

          right  bottom  
block                    
30961   13.8985    4.86  
34405  118.1025    5.31  
35975   39.1865    5.58  
40284  101.9935    6.21  
41318   31.4940    6.39  


#### General functions

In [3]:
#get number of days between and two dates; used to convert dates to 1-366 day of year 
def dayDiff(start,end):
  startdate = datetime.datetime.strptime(start,"%Y%m%d")
  enddate = datetime.datetime.strptime(end,"%Y%m%d")
  days = enddate-startdate
  return (days.days+1)

In [4]:
#DIST-S1 generate dictionary of daily STATUS values per ID (note switched path to block instead of MGRS tile)
def getDISTS1status_vI(block):#,skipNodata=False):
    #print(skipNodata)
    mapalert = {}
    IDlist = [ID for ID in allIDs if block in ID]

    for ID in IDlist:
        mapalert[ID] = [255 for i in range(0,366)]
        #print(ID,end=',')
        with open(mapsourceS1+'/'+block+'/'+ID+'.csv','r') as mapfile:
            lines = mapfile.readlines()
            header = lines[0]
            maplist = lines[1:]
            for line in maplist:
                try:
                    (temp,SensingTime,STATUS)= line.strip().split(',')
                    day = dayDiff("20240101",datetime.datetime.strftime(datetime.datetime.strptime(SensingTime,"%Y-%m-%d"),"%Y%m%d"))

                    #if not (skipNodata and VEGANOM!='NA'):
                    mapalert[ID][day] = int(STATUS)
                except:
                #    print(traceback)
                    print(ID,line)

    return mapalert

In [5]:
#generate dictionary of ref no, low, high change and no data for each day of year (note conversion only and only 2024 parameters don't work)
def getRefALERTDaily(filename,high=["VLmaj"],low=["VLmin"],nochange=["OCmin","OCmaj","VGmin","VGmaj","noChange"],IDlist=allIDs,conversiononly=False,only2024=False):
  #if conversiononly or only2024:
  #  with open("reference_conversion.csv","r") as reffile:
  #    reflist = reffile.readlines()[1:]
  #  refconv = {}
  #  refprevyear = {}
  #  natural = {}
  #  for line in reflist:
  #    fields = line.strip().split(",")
  #    (ID,changetype,conversion,naturalproportion,prevyear,overallLabel)=fields[0:6]
  #    refconv[ID]=conversion
  #    refprevyear[ID]=prevyear
  #    natural[ID] = naturalproportion
  refalert = {}
  with open(filename,"r") as mapfile:
    lines = mapfile.readlines()
    header = lines[0]
    reflist = lines[1:]
  for line in reflist:
    fields = line.strip().split(",")
    (ID,overallLabel,Long,Lat,changetype) = fields[0:5]
    refalert[ID] = [0 for i in range(0,366)]
    if ID in IDlist:
      daily = fields[5:]
      #refalert[ID] = [0 for i in range(0,366)]
      try:
        for day in range(0,366):
          found = False
          for l in high:
            if l == daily[day]:
              refalert[ID][day] = 3
          for l in low:
            if l == daily[day]:
              refalert[ID][day] = 2
          for l in nochange:
            if l == daily[day]:
              refalert[ID][day] = 1
          #if conversiononly and (refconv[ID] != "natural" and (refconv[ID] != "human" or (refconv[ID] == "human" and natural[ID] == '0'))):#(refconv[ID] != "human" or (refconv[ID] == "human" and natural[ID] == '0')):#(refconv[ID] == "no" or natural[ID] == '0'):
          #  refalert[ID][day] = 0
          #if only2024 and refprevyear[ID] == "TRUE":
          #  refalert[ID][day] = 0
      except:
        print(ID,day,daily)
  return refalert

In [6]:
#build confusion matrix for a block of no, low, and high change
def getMatrixBlock(block,mapin,maplow,maphigh,nodata=[255],refType="VL",convOnly=False,only24 =False,mincount=10,Ndays=30):
  strataList=[1,2,3,4]
  strataDict=substrataDict
  mapout = {}
  n = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in strataList}
  ntotal = {s:0 for s in strataList}
  if refType == "VL":
    ref = getRefALERTDaily(referenceSource,high=["VLmaj"],low=["VLmin","VLsub"],nochange=["VGmin","VGmaj","OCmin","OCmaj","noChange","VGsub","OCsub"],conversiononly=convOnly,only2024=only24)
  elif refType =="VG":
    ref = getRefALERTDaily(referenceSource,high=["VGmaj"],low=["VGmin"],nochange=["noChange","VLmin","VLmaj","OCmin","OCmaj","VLsub","OCsub"],conversiononly=convOnly,only2024=only24)  
  elif refType =="OC":
    ref = getRefALERTDaily(referenceSource,high=["OCmaj"],low=["OCmin"],nochange=["noChange","VLmin","VLmaj","VGmin","VGmaj","VLsub","VGsub"],conversiononly=convOnly,only2024=only24)  
  elif refType =="ALL":
    ref = getRefALERTDaily(referenceSource,high=["VLmaj","VGmaj","OCmaj"],low=["VLmin","VGmin","OCmin"],nochange=["noChange"],conversiononly=convOnly,only2024=only24)

  IDlist = [ID for ID in list(ref.keys()) if block in ID]

  #confusion matrix
  for ID in IDlist:
    stratum = strataDict[ID]
    #if stratum in selectedStrata:
    try:
        p = [[0,0,0],[0,0,0],[0,0,0]]
        ptotal = 0
        mapout[ID] = [0 for x in range(0,366)]
        for d in range(0,366):
          if mapin[ID][d] in [255] or mapin[ID][d] in nodata:
              mapout[ID][d] = 0
          elif mapin[ID][d] in [0]:
              mapout[ID][d] = 1
          elif mapin[ID][d] in maplow:
              mapout[ID][d] = 2
          elif mapin[ID][d] in maphigh:
              mapout[ID][d] = 3
          else:############added to exclude fron matrix but include in proportion
              mapout[ID][d] = 4
          #if not int(ID) in excludelist:
          if max(ref[ID][0:(d+1)])>0 and mapout[ID][d] != 0:
                start = (d>Ndays)*(d-Ndays)

                #current anomoly/no anomaly; compare against lookback window
                if mapin[ID][d] < 7:
                    if ref[ID][start:(d+mincount)].count(2)+ref[ID][start:(d+mincount)].count(3) > mincount:
                      if ref[ID][start:(d+mincount)].count(3) > 0:
                        refVal=3
                      else:
                        refVal=2
                    elif ref[ID][start:(d+1)].count(1) > 0:
                        refVal=1
                    else:
                        refVal=0
                        
                #finished anomaly; compare year to date
                elif mapin[ID][d] >= 7 and mapin[ID][d]!=255:
                    start = 0
                    if ref[ID][start:(d+mincount)].count(2)+ref[ID][start:(d+mincount)].count(3) > mincount:
                      if ref[ID][start:(d+mincount)].count(3) > 0:
                        refVal=3
                      else:
                        refVal=2
                    elif ref[ID][start:(d+1)].count(1) > 0:
                        refVal=1
                    else:
                        refVal=0
                #nodata
                else:
                  refVal=0
                mapVal = mapout[ID][d]
                if mapVal==4 and refVal>0:
                  ptotal += 1
                elif refVal>0 and mapVal>0:
                    p[refVal-1][mapVal-1] += 1
                    ptotal += 1
        if ptotal>0:
          ntotal[stratum] += 1
          for r in [0,1,2]:
            for m in [0,1,2]:
              n[stratum][r][m] += (p[r][m]/ptotal)
    except:
        print(ID,"missing",stratum,d,p,ptotal,ntotal[stratum])
        print(mapin[ID])
        print(ref[ID])

  return (n,ntotal)

In [7]:
#convert matrix from three classes (no, low, high) to two classes (no, yes) for different accuracy metrics
def convMat(n,selectedStrata=[1,2,3,4]):
  nlowuser = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in selectedStrata}
  nlowprod = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in selectedStrata}
  nhiuser = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in selectedStrata}
  nhiprod = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in selectedStrata}
  nalluser = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in selectedStrata}
  nallprod = {s:[[0,0,0],[0,0,0],[0,0,0]] for s in selectedStrata}
  NO = 0
  LOW = 1
  HI = 2
  for s in selectedStrata:
    stringS = str(s)

    #[stratum][ref][map]
    nlowprod[s][2][2] = n[s][LOW][LOW] + n[s][LOW][HI]
    nlowprod[s][2][1] = n[s][LOW][NO]
    nlowuser[s][2][2] = n[s][LOW][LOW] + n[s][HI][LOW]
    nlowuser[s][1][2] = n[s][NO][LOW]

    nhiprod[s][2][2] = n[s][HI][HI] + n[s][HI][LOW]
    nhiprod[s][2][1] = n[s][HI][NO]
    nhiuser[s][2][2] = n[s][HI][HI] + n[s][LOW][HI]
    nhiuser[s][1][2] = n[s][NO][HI]

    nallprod[s][2][2] = n[s][HI][HI] + n[s][HI][LOW] + n[s][LOW][HI] + n[s][LOW][LOW]
    nallprod[s][2][1] = n[s][HI][NO] + n[s][LOW][NO]
    nalluser[s][2][2] = n[s][HI][HI] + n[s][LOW][HI] + n[s][HI][LOW] + n[s][LOW][LOW]
    nalluser[s][1][2] = n[s][NO][HI] + n[s][NO][LOW]
  return (nlowuser, nlowprod, nhiuser, nhiprod,nalluser, nallprod)

In [8]:
#compute users accuracy for a block and return stat and SE
def usersAccuracyBlock(n, ntotal, block):
  N = selectedBlocks.loc[int(block),['sub1','sub2','sub3','sub4']] ## create tables of substrata areas
  #Accuracy
  y = 0
  usersx = 0
  for s in [1,2,3,4]:
    if ntotal[s]>0:
      y += (n[s][2][2]/ntotal[s])*N.iloc[s-1]
      usersx += ((n[s][1][2]+n[s][2][2])/ntotal[s])*N.iloc[s-1]
  if usersx > 0:
    users = (y/usersx)
    usersxout = usersx
  else:
    users = "NA"
    usersSE = "NA"
    usersxout = None
  yout = y

  UAsub1 = 0
  UAsub2 = 0
  if users != "NA":
    for s in [1,2,3,4]:
      if (n[s][1][2]+n[s][2][2]) > 0 and ntotal[s]>1:
        yhmean = n[s][2][2]/ntotal[s]
        yhsampvar = ((n[s][2][2])*((1-yhmean)**2) + (n[s][1][1] + n[s][1][2] + n[s][2][1])*((0-yhmean)**2))/(ntotal[s]-1)
        xhmean = (n[s][1][2]+n[s][2][2])/ntotal[s]
        xhsampvar = ((n[s][1][2]+n[s][2][2])*((1-xhmean)**2) + (n[s][1][1] + n[s][2][1])*((0-xhmean)**2))/(ntotal[s]-1)
        xyhsampvar = (n[s][1][1] * (0-yhmean) * (0-xhmean) + n[s][1][2] * (0-yhmean) * (1-xhmean) + n[s][2][1] * (0-yhmean) * (0-xhmean) + n[s][2][2] * (1-yhmean) * (1-xhmean))/(ntotal[s] - 1)
        UAsub1 += N.iloc[s-1]*xhmean
        UAsub2 += N.iloc[s-1]**2 * (1 - ntotal[s]/N.iloc[s-1]) * (yhsampvar + (users**2)*xhsampvar - 2*users*xyhsampvar)/ntotal[s]
  
  if users != "NA":
    if UAsub1>0 and UAsub2>0:
        usersSE = math.sqrt(1/(UAsub1**2) * UAsub2)
    else:
        usersSE = None
    users = users
  else:
    users = None
    usersSE = None
  return [users,usersSE,yout,usersxout]

In [9]:
#compute producers accuracy for a block and return stat and SE
def producersAccuracyBlock(n, ntotal, block):
  N = selectedBlocks.loc[int(block),['sub1','sub2','sub3','sub4']] ## create tables of substrata areas
  #Accuracy
  y = 0
  producersx = 0
  for s in [1,2,3,4]:
    if ntotal[s]>0:
      y += (n[s][2][2]/ntotal[s])*N.iloc[s-1]
      producersx += ((n[s][2][1]+n[s][2][2])/ntotal[s])*N.iloc[s-1]

  if producersx > 0:
    producers = (y/producersx)
    prodxout = producersx
  else:
    producers = "NA"
    producersSE = "NA"
    prodxout = None
  yout = y

  PAsub1 = 0
  PAsub2 = 0
  for s in [1,2,3,4]:
    if producers != "NA":
        if (n[s][2][1]+n[s][2][2]) > 0 and ntotal[s]>1:
            yhmean = n[s][2][2]/ntotal[s]
            yhsampvar = ((n[s][2][2])*((1-yhmean)**2) + (n[s][1][1] + n[s][1][2] + n[s][2][1])*((0-yhmean)**2))/(ntotal[s]-1)
            xphmean = (n[s][2][1]+n[s][2][2])/ntotal[s]
            xphsampvar = ((n[s][2][1]+n[s][2][2])*((1-xphmean)**2) + (n[s][1][1] + n[s][1][2])*((0-xphmean)**2))/(ntotal[s]-1)
            xyphsampvar = (n[s][1][1] * (0-yhmean) * (0-xphmean) + n[s][1][2] * (0-yhmean) * (0-xphmean) + n[s][2][1] * (0-yhmean) * (1-xphmean) + n[s][2][2] * (1-yhmean) * (1-xphmean))/(ntotal[s] - 1)
            PAsub1 += N.iloc[s-1]*xphmean
            PAsub2 += N.iloc[s-1]**2 * (1 - ntotal[s]/N.iloc[s-1]) * (yhsampvar + (producers**2)*xphsampvar - 2*producers*xyphsampvar)/ntotal[s]
  
  if producers != "NA":
    if PAsub1 >0 and PAsub2>0:
      producersSE = math.sqrt(1/(PAsub1**2) * PAsub2)
    else:
      producersSE = None
    producers = producers
  else:
    producers = None
    producersSE = None

  return [producers,producersSE,yout,prodxout]

In [10]:
def getAccuracies(n,ntotal, name,block, measure="both"):
  (nlowuser, nlowprod, nhiuser, nhiprod,nalluser, nallprod) = convMat(n)
  loU="NA"
  loUSE="NA"
  loP="NA"
  loPSE="NA"
  hiU="NA"
  hiUSE="NA"
  hiP="NA"
  hiPSE="NA"
  aU="NA"
  aUSE="NA"
  aP="NA"
  aPSE="NA"
  if measure =="both" or measure == "users":
    (loU,loUSE,y,x) = usersAccuracyBlock(nlowuser, ntotal, block)
    (hiU,hiUSE,y,x) = usersAccuracyBlock(nhiuser, ntotal, block)
    (aU,aUSE,y,x) = usersAccuracyBlock(nalluser, ntotal, block)
  if measure =="both" or measure =="producers":
    (loP,loPSE,y,x) = producersAccuracyBlock(nlowprod, ntotal, block)
    (hiP,hiPSE,y,x) = producersAccuracyBlock(nhiprod, ntotal, block)
    (aP,aPSE,y,x) = producersAccuracyBlock(nallprod, ntotal, block)
  return [[name+"_low",loU,loUSE,loP,loPSE],[name+"_high",hiU,hiUSE,hiP,hiPSE],[name+"_all",aU,aUSE,aP,aPSE]]

## Accuracy 

#### Calculate accuracy single block

Accuracy for DIST-S1 for detecting vegetation loss for one block (Users SE Producers SE)

In [11]:
block = str(34405)
print("Accuracies for",block,"from stratum",blockstrataDict[block])
map=getDISTS1status_vI(block)
accuracies = []
(n,ntotal)=getMatrixBlock(block,map,[2],[5],nodata=[1,3,7,4,6,8],refType="VL")
accuracies = accuracies + getAccuracies(n,ntotal, "prov",block,measure="users")

(n,ntotal)=getMatrixBlock(block,map,[2,3,7],[5,6,8],nodata=[1,4],refType="VL")
accuracies = accuracies + getAccuracies(n,ntotal, "provconf",block)

(n,ntotal)=getMatrixBlock(block,map,[3,7],[6,8],nodata=[1,2,4,5],refType="VL")
accuracies = accuracies + getAccuracies(n,ntotal, "conf",block)

#(n,ntotal)=getMatrixBlock(block,map,[2,3,7],[5,6,8],nodata=[1,4],convOnly=True,only24=True)
#accuracies = accuracies + getAccuracies(n,ntotal, "conversion",block,measure="producers")

accuracies = pd.DataFrame(accuracies,columns=["name","users","usersSE","producers","producersSE"])
accuracies = accuracies[["users","usersSE","producers","producersSE"]].set_index(accuracies.name)
print(accuracies)

Accuracies for 34405 from stratum treelosswet
               users       usersSE producers producersSE
name                                                    
prov_low         1.0           NaN        NA          NA
prov_high        1.0  1.040352e-08        NA          NA
prov_all         1.0  6.434231e-09        NA          NA
provconf_low     1.0  1.025124e-08  0.337085    0.279563
provconf_high    1.0           NaN  0.731389     0.17534
provconf_all     1.0           NaN  0.626946    0.155653
conf_low         1.0           NaN  0.263579    0.252484
conf_high        1.0           NaN  0.702895    0.181774
conf_all         1.0           NaN  0.588562    0.160758


Accuracy for DIST-S1 for detecting all change for one block (Users SE Producers SE)

In [12]:
block = str(34405)
map=getDISTS1status_vI(block)
accuracies = []
(n,ntotal)=getMatrixBlock(block,map,[2],[5],nodata=[1,3,7,4,6,8],refType="ALL")
accuracies = accuracies + getAccuracies(n,ntotal, "prov",block,measure="users")

(n,ntotal)=getMatrixBlock(block,map,[2,3,7],[5,6,8],nodata=[1,4],refType="ALL")
accuracies = accuracies + getAccuracies(n,ntotal, "provconf",block)

(n,ntotal)=getMatrixBlock(block,map,[3,7],[6,8],nodata=[1,2,4,5],refType="ALL")
accuracies = accuracies + getAccuracies(n,ntotal, "conf",block)

#(n,ntotal)=getMatrixBlock(block,map,[2,3,7],[5,6,8],nodata=[1,4],convOnly=True,only24=True)
#accuracies = accuracies + getAccuracies(n,ntotal, "conversion",block,measure="producers")

accuracies = pd.DataFrame(accuracies,columns=["name","users","usersSE","producers","producersSE"])
accuracies = accuracies[["users","usersSE","producers","producersSE"]].set_index(accuracies.name)
print(accuracies)

               users       usersSE producers producersSE
name                                                    
prov_low         1.0           NaN        NA          NA
prov_high        1.0           NaN        NA          NA
prov_all         1.0  6.291432e-09        NA          NA
provconf_low     1.0  1.055966e-08  0.372915    0.290788
provconf_high    1.0           NaN  0.734387    0.174328
provconf_all     1.0           NaN  0.642107    0.154299
conf_low         1.0           NaN  0.305378    0.270588
conf_high        1.0           NaN  0.706145    0.180746
conf_all         1.0           NaN  0.606104    0.159683


#### Calculate accuracy for all blocks

In [13]:
def getBlockAccuracy(block,measure,changeintensity,maplowclasses,maphighclasses,nodataclasses,refChangeType):
  """Calculates the accuracy for the given block
  Args:
    block: (str) block ID
    measure: "users" or "producers"
    changeintensity: "low", "high", or "all" (defines what intensity threshold is evaluated for statistic)
    maplowclasses: [int] ( the status classes that will be marked as low intensity, e.g. for confirmed only [3,7])
    maphighclasses: [int] ( the status classes that will be marked as high intensity, e.g. for confirmed only [6,8])
    nodataclasses: [int] ( the status classes that will be marked as no data/excluded, e.g. for confirmed only [1,2,4,5,255] in order for the first and provisional to neither be counted as right or wrong)
    refChangeType: "VL", "VG", "OC", "ALL" (sets what type of reference change the product is evaluated against)
  """
  map=getDISTS1status_vI(block)
  (n,ntotal)=getMatrixBlock(block,map,maplowclasses,maphighclasses,nodataclasses,refType=refChangeType)
  (nlowuser, nlowprod, nhiuser, nhiprod,nalluser, nallprod) = convMat(n)
  if measure == "users":
    if changeintensity == "low":
      return usersAccuracyBlock(nlowuser, ntotal, block)  #returns (users,usersSE,yout,usersxout)
    if changeintensity == "high":
      return usersAccuracyBlock(nhiuser, ntotal, block)  #returns (users,usersSE,yout,usersxout)
    if changeintensity == "all":
      return usersAccuracyBlock(nalluser, ntotal, block)  #returns (users,usersSE,yout,usersxout)
  if measure =="producers":
    if changeintensity == "low":
      return producersAccuracyBlock(nlowprod, ntotal, block)  #returns (producers,producersSE,yout,prodxout)
    if changeintensity == "high":
      return producersAccuracyBlock(nhiprod, ntotal, block)  #returns (producers,producersSE,yout,prodxoutE)
    if changeintensity == "all":
      return producersAccuracyBlock(nallprod, ntotal, block)  #returns (producers,producersSE,yout,prodxout)

Per accuracy statistic: (1) estimate mean and standard error per block stratum, (2) estimate the global accuracies and standard errors.

In [14]:
#test
print(getBlockAccuracy(str(34405),"producers",changeintensity="high",maplowclasses=[3,7],maphighclasses=[6,8],nodataclasses=[255,1,2,4,5],refChangeType="ALL"))

[0.7061447606181053, 0.18074645695372377, 3651.164140483357, 5170.560406463126]


In [15]:
def strataRatioAccuracy(stratum,measure,changeintensity,maplowclasses,maphighclasses,nodataclasses,refChangeType,printdf=False):
  blocks = getBlocksStratum(stratum)
  statBlocks = {}
  SEBlocks = {}
  #nh = len(blocks)
  Nh = scounts.loc[stratum]['blockCount']
  y = {}
  x={}
  Xhat = 0
  for block in blocks:
    if block != 488254:
      block = str(block)
      (statBlocks[block],SEBlocks[block],y[block],x[block]) = getBlockAccuracy(str(block),measure,changeintensity,maplowclasses,maphighclasses,nodataclasses,refChangeType)
  df = pd.DataFrame.from_dict(statBlocks,orient='index')
  df.columns = ['measure']
  df['yu']=y
  df['xu']=x
  nh = df['measure'].count()
  if nh > 1:
    measure = df['yu'].sum()/df['xu'].sum() #* 100 # statistics.mean(df['measure'])
    yhat = (df['yu'].sum()/nh)
    xhat = (df['xu'].sum()/nh)
    if printdf:
      print(stratum)
      print(df)
    yvar = ((df['yu']**2).sum() - nh*(yhat**2))/(nh - 1)
    xvar = ((df['xu']**2).sum() - nh*(xhat**2))/(nh - 1)
    xyvar = ((df['yu']*df['xu']).sum() - nh*yhat*xhat)/(nh - 1)
    #variance = (1/(Nh*xhat)**2) * (Nh**2 *(1-nh/Nh) * (yvar + (measure**2) * xvar - 2*measure*xyvar) ) / nh
    subvariance = (Nh**2 *(1-nh/Nh) * (yvar + (measure**2) * xvar - 2*measure*xyvar) ) / nh
    SE = math.sqrt((1/(Nh*xhat))**2 * subvariance) #* 100
    return(measure,SE,subvariance,nh,yhat,xhat)#,yvar,xvar,xyvar) #SE and variance does not take into account within block variance
  else:
    raise Exception("less than 2 blocks")

In [16]:
def globalRatioAccuracy(measure,changeintensity,maplowclasses,maphighclasses,nodataclasses,refChangeType="ALL"):
  statStrata = {}
  SEstrata = {}
  Vstrata = {}
  nh = {}
  Nh = {}
  strataList = allStrata
  successfulStrata = []
  failedStrata = []
  yhat = {}
  xhat = {}
  for s in strataList:
    #print(s)
    try:
      (statStrata[s],SEstrata[s],Vstrata[s],nh[s],yhat[s],xhat[s]) = strataRatioAccuracy(s,measure,changeintensity,maplowclasses,maphighclasses,nodataclasses,refChangeType)
      Nh[s] = scounts.loc[s]['blockCount']
      successfulStrata.append(s)
    except:
      failedStrata.append(s)
  print("successful",successfulStrata)
  print("failed",failedStrata)
  df = pd.DataFrame.from_dict(statStrata,orient='index')
  df.columns=['measure']
  df['SE'] = SEstrata
  df['SVar'] = Vstrata
  df['nh'] = nh
  df['Nh'] = Nh
  df['yhat'] = yhat
  df['xhat'] = xhat
  globalMeasure = (df['yhat']*df['Nh']).sum()/(df['xhat']*df['Nh']).sum()
  totalBlocksOfStrata = df['Nh'].sum()
  bigXhat = (df['xhat']*df['Nh']).sum()
  globalVar = df['SVar'].sum() /(bigXhat**2)
  globalSE = math.sqrt(globalVar)
  print(df)
  return(globalMeasure,globalSE,globalVar,df)

In [17]:
(stat,SE,var,df) = globalRatioAccuracy("users",changeintensity="high",maplowclasses=[3,7],maphighclasses=[6,8],nodataclasses=[255,1,2,4,5],refChangeType="ALL") #confirmed
print("global users confirmed",stat,'±',SE)

(stat,SE,var,df) = globalRatioAccuracy("producers",changeintensity="high",maplowclasses=[3,7],maphighclasses=[6,8],nodataclasses=[255,1,2,4,5],refChangeType="ALL") #confirmed
print("global producers confirmed",stat,'±',SE)

(stat,SE,var,df) = globalRatioAccuracy("users",changeintensity="high",maplowclasses=[2,3,7],maphighclasses=[5,6,8],nodataclasses=[255,1,5],refChangeType="VL") #provisional and confirmed
print("global users provisional and confirmed",stat,'±',SE)

(stat,SE,var,df) = globalRatioAccuracy("producers",changeintensity="high",maplowclasses=[2,3,7],maphighclasses=[5,6,8],nodataclasses=[255,1,4],refChangeType="ALL") #provisional and confirmed
print("global producers provisional and confirmed",stat,'±',SE)

successful []
failed ['waternew', 'treelosswet', 'builtnewalert', 'fire', 'treelossTF', 'cropnew', 'wetshort', 'oldcrop_short', 'gen', 'other', 'none']


ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

## DIST-ALERT-HLS functions

In [17]:
#DIST-ALERT-HLS
def getDISTALERTStatus_vI(block,skipNodata=False):
    #print(skipNodata)
    mapalert = {}
    IDlist = [ID for ID in allIDs if str(block) in ID]

    for ID in IDlist:
        mapalert[ID] = [255 for i in range(0,367)]
        #print(ID,end=',')
        with open(mapsourceHLS+'/'+ID+'_DIST-ALERT_'+ANNname+'.csv','r') as mapfile:
            lines = mapfile.readlines()
            header = lines[0]
            maplist = lines[1:]
            for line in maplist:
                try:
                    (granuleID,SensingTime,ProductionTime,VEGDISTSTATUS,VEGANOM,VEGIND,VEGHIST,VEGANOMMAX,VEGDISTCONF,VEGDISTDATE,VEGDISTCOUNT,VEGDISTDUR,VEGLASTDATE,GENDISTSTATUS,GENANOM,GENANOMMAX,GENDISTCONF,GENDISTDATE,GENDISTCOUNT,GENDISTDUR,GENLASTDATE)= line.strip().split(',')
                    day = dayDiff("20240101",SensingTime[0:8])

                    if not (skipNodata and VEGANOM!='NA'):
                        mapalert[ID][day] = int(VEGDISTSTATUS)
                except:
                #    print(traceback)
                    print(ID,day,line)

    return mapalert


In [18]:
def getDISTALERTStatus_vI_GEN(skipNodata=False):
    #print(skipNodata)
    mapalert = {}
    for ID in allIDs:
        mapalert[ID] = [255 for i in range(0,367)]
        #print(ID,end=',')
        with open(mapsourceHLS+'/'+ID+'_DIST-ALERT_'+ANNname+'.csv','r') as mapfile:
            lines = mapfile.readlines()
            header = lines[0]
            maplist = lines[1:]
            for line in maplist:
                try:
                    (granuleID,SensingTime,ProductionTime,VEGDISTSTATUS,VEGANOM,VEGIND,VEGHIST,VEGANOMMAX,VEGDISTCONF,VEGDISTDATE,VEGDISTCOUNT,VEGDISTDUR,VEGLASTDATE,GENDISTSTATUS,GENANOM,GENANOMMAX,GENDISTCONF,GENDISTDATE,GENDISTCOUNT,GENDISTDUR,GENLASTDATE)= line.strip().split(',')
                    day = dayDiff("20230101",SensingTime)

                    if not (skipNodata and int(GENANOM)==255):
                      mapalert[ID][day] = int(GENDISTSTATUS)
                except:
                #    print(traceback)
                    print(ID,line)

    return mapalert